## Notes

Need to add:
- event date and name (for calculating fighter age and time since last fight)
- Filter out upcoming fights (do not scrape that) 

In [1]:
# Importing 
import requests
import csv 
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from google.colab import files
import datetime
from datetime import date
from pytz import timezone
eastern = timezone('US/Eastern')
import threading 
from concurrent.futures import ThreadPoolExecutor
import multiprocessing
cores = multiprocessing.cpu_count()

In [ ]:
# MAIN PROGRAM 
master_data = [] 
# Two main scraping programs 
fighter_urls = get_fighter_urls()
master_data = scrape_fights(fighter_urls)  # takes 8 minutes on just the "A" list 

# --------------------------------------------------------------------------------------------------------
def concurrent_ufc_fights(ufcFighterLinks, increments):
  start_time = time.time()
  ufcFights = dict()

  with ThreadPoolExecutor(max_workers = increments) as executor:
    for fightLink in ufcFighterLinks:
      try:
      # get the data and add to a master data file
        ufcEvents[fightLink] = executor.submit(scrape_fights, fightLink) # get one fighters detail 
        # ufcEvents.append(eventData)
      except:
        print("Error on link: ", fightLink)  
  
  completion_time = time.time() - start_time
  print("Time to complete: ", completion_time)
  to_return = {fightLink : future.result() for fightLink, future in ufcFights.items()}
  return to_return


fights = concurrent_ufc_fights(fighter_urls, 8) 
ufcFights = []

# put all the events in the ufcFights dataset 
for url in events:
  row = events[url]
  ufcFights.append(row)  
# --------------------------------------------------------------------------------------------------------

# Now create the master dataset 
# master_data
master_data_columns = pd.read_excel('/content/drive/MyDrive/UFC Model/Data/fight_columns.xlsx', header=0)
columns_list = list(master_data_columns.columns.values)
pd_data = pd.DataFrame(master_data, columns = columns_list)
# Export to Drive and Download
pd_data.to_excel('/content/drive/MyDrive/UFC Model/ufcFights.xlsx')
pd_data.to_csv('/content/drive/MyDrive/UFC Model/ufcFights.csv')
# files.download('/content/drive/MyDrive/UFC Model/ufcFights.xlsx')

In [ ]:
fighter_urls

In [3]:
def get_fighter_urls():
  # get a massive list of fighter URLs 
  fighter_url_links = pd.read_csv('/content/drive/MyDrive/UFC Model/Data/fighterLinks.csv', header=0)
  fighter_urls = fighter_url_links.values.tolist()

  # Get HTML from page 
  fighter_list  = 'http://ufcstats.com/statistics/fighters?char=a&page=all'  # THIS IS THE TEST 
  fighter_list_page = requests.get(fighter_list)
  fighter_list_soup = BeautifulSoup(fighter_list_page.text, 'html.parser')

  fighterLink = 'http://ufcstats.com/fighter-details/' # UFC Fighter substring link
  fighter_findUrls = fighter_list_soup.find_all('a')

  # Find urls on fighter profile page 
  fighter_urls = []
  for link in fighter_findUrls:
      pageLink = str(link.get('href'))
      fighter_urls.append(pageLink)

  # filter out fight links to loop through 
  fighterUrls = []
  counter = 0 
  for theLink in fighter_urls:
    if fighterLink in theLink:
      if theLink not in fighterUrls:
        fighterUrls.append(theLink)
        # print(theLink)
        counter += 1
    else: continue
      # print('Filtered Out: ', theLink)

  return fighterUrls


In [4]:
# start_time = time.time()
def scrape_fights(fighterUrls):
  
  for fighter_url in fighterUrls: 
    fighter_page  = requests.get(fighter_url)
    fighter_soup  = BeautifulSoup(fighter_page.text, 'html.parser')
    fighter_title = fighter_soup.find_all('li', {'class' : "b-list__box-list-item b-list__box-list-item_type_block"})

    # Get fighter full name 
    name = fighter_soup.find_all('span', {'class' : 'b-content__title-highlight'})
    for item in name:  
      fighterName = item.text.strip()
      # print(fighterName)

    nickName = fighter_soup.find_all('p', {'class' : "b-content__Nickname"}) 
    for item in nickName:
      nickname = item.text.strip()
      if nickname == '':
        nickname = '--'
            
    # Get basic fighter information 
    for item in fighter_title:
      
      word = item.text.strip()

      if 'Height:' in word: 
          height = word.replace('Height:', '').strip()    
      
      elif 'Weight:' in word: 
          weight = word.replace('Weight:', '').strip()
      
      elif 'DOB:' in word:
          dob =    word.replace('DOB:', '').strip()
          # dob =    word.replace(",", "").strip()
      
      elif 'STANCE' in word:
          stance = word.replace('STANCE:', '').strip()
          if stance == '' or stance ==' ':
            stance == '--'
      
      elif 'Reach:' in word:
          reach =  word.replace('Reach:', '').strip()
          if reach == '' or reach==' ':
            reach == '--'

      else:
        continue

    # Create base fighter profile 
    fighter_profile_base = []
    fighter_profile_base.append(fighter_url)
    fighter_profile_base.append(fighterName)
    fighter_profile_base.append(nickname)
    fighter_profile_base.append(dob)
    fighter_profile_base.append(height)
    fighter_profile_base.append(weight)
    fighter_profile_base.append(reach) 
    fighter_profile_base.append(stance)

    print('Parsing: ', fighterName)

    # get each fight URL from a fighter page
    fightLink = 'http://ufcstats.com/fight-details/' # UFC Fighter substring link

    findUrls = fighter_soup.find_all('a')
    # Find urls on fighter profile page 
    urls = []
    for link in findUrls:
        pageLink = str(link.get('href'))
        urls.append(pageLink)

    # filter out fight links to loop through 
    fightUrls = []
    counter = 0 
    for theLink in urls:
      if fightLink in theLink:
        if theLink not in fightUrls:
          fightUrls.append(theLink)
          # print(theLink)
          counter += 1
      else: continue
        # print('Filtered Out: ', theLink)
    # print()    
    # print('Number of Fights: ', counter)

    fight_array = []

    for fight_url in fightUrls:

      # Set a fight row to begin with basic profile information (for every row on a fighter)
      fight_row = []

      fight_page = requests.get(fight_url)
      fight_soup = BeautifulSoup(fight_page.text, 'html.parser')

      # get event title 
      event_title = fight_soup.find_all('h2', {'class' : 'b-content__title'})
      
      for item in event_title:
        event = item.text.strip()
        # print('Fight:', fight_url)
        # print('Event:', event)
        fight_row.append(fight_url)
        fight_row.append(event)

      fight_outcomes = fight_soup.find_all("div", {'class' : 'b-fight-details__person'})

      for fighter in fight_outcomes:
        opponent = False
        results = fighter.findAll('i')
        links   = fighter.findAll('a')
      
        for a in links:
              name = a.text.strip()
              if name != fighterName:
                opponent = True
                opponentName = name
                opponentLink = str(a.get('href'))

        for result in results:
          fight_result  = result.text.strip()
          if not opponent:
            fighter_result = fight_result 
          else:
            opponent_result = fight_result  

      # print('Parsing... ',fighterName, ' vs. ', opponentName)
      #print('Opponent: ', opponentName)
      #print('Opponent Link: ', opponentLink)
      #print('Fighter Result: ', fighter_result)
      #print('Opponent Result:', opponent_result)

      fight_row.append(fighterName)
      fight_row.append(opponentName)
      fight_row.append(opponentLink)
      fight_row.append(fighter_result)
      fight_row.append(opponent_result)
      
      fight_title = fight_soup.find_all('i', {'class' : "b-fight-details__fight-title"})

      for item in fight_title:
        type_of_fight = item.text.strip()
        # print(type_of_fight)
        fight_row.append(type_of_fight)
        # fight, perf, sub, ko (bonuses)
        # another indicator for a very good past performance judged by company...
        bonuses = item.findAll('img')
        # intialize variables
        belt = "-"
        perf = "-"
        fon  = "-"
        sub  = "-"
        ko   = '-'

        for bonus in bonuses:
          if 'belt.png' in bonus['src']:  # "title bout" also included in the title 
            belt = 1 
            # print("Championship Fight")
          if 'perf.png' in bonus['src']:
            perf = 1
            # print('Bonus: Performance of the night')
          if 'fight.png' in bonus['src']: 
            fon = 1
            # print('Bonus: Fight of the Night')
          if 'sub.png' in bonus['src']:
            sub = 1 
            # print('Bonus: Submission of the Night')  
          if 'ko.png' in bonus['src']: 
            ko = 1 
            # print('Bonus: KO of the Night')  

      fight_row.append(belt)
      fight_row.append(perf)
      fight_row.append(fon)
      fight_row.append(sub)
      fight_row.append(ko)
      
      # Fight statistics 
      fight_table  = fight_soup.find_all(True, {'class' : ["b-fight-details__text-item", "b-fight-details__text-item_first"]})

      detail_section = False
      det = ' '

      # Basic fight summary statistics 
      for item in fight_table:
          word = item.text.strip()
          word =' '.join(word.split())
          if 'Method:' in word: 
              method = word.replace('Method:', '').strip()
          if 'Round:' in word: 
              round = word.replace('Round:', '').strip()
          if 'Time:' in word:
              time = word.replace('Time:', '').strip()
          if 'Time format:' in word:
              timeFormat = word.replace('Time format:', '').strip() 

      # print(method)
      # print(round)
      # print(time)
      # print(timeFormat)
      fight_row.append(method)
      fight_row.append(round)
      fight_row.append(time)
      fight_row.append(timeFormat)

      fight_details  = fight_soup.find_all(True, {'class' : "b-fight-details__text"})

      for item in fight_details:
        word = item.text.strip()
        word = ' '.join(word.split())
        if 'Details:' in word:
          details = ' '.join(word.split()) 
          details = details.replace('Details:', '')
          details = details.strip()
        else:
          details = '--'
          
      fight_row.append(details)
      full_row = fighter_profile_base + fight_row

      master_data.append(full_row)
  
  return master_data 


In [ ]:
def concurrent_ufc_fights(ufcFighterLinks, increments):
  start_time = time.time()
  ufcFights = dict()

  with ThreadPoolExecutor(max_workers = increments) as executor:
    for fightLink in ufcFighterLinks:
      try:
      # get the data and add to a master data file
        ufcEvents[eventLink] = executor.submit(get_fight_data, fightLink)
        # ufcEvents.append(eventData)
      except:
        print("Error on link: ", fightLink)  
  
  completion_time = time.time() - start_time
  print("Time to complete: ", completion_time)
  to_return = {fightLink : future.result() for fightLink, future in ufcFights.items()}
  return to_return

  # "Fights" is the result of running concurrent program to scrape all fights on the site
fights = concurrent_ufc_fights(ufcFighterLinks, 8) # use 8 workers for ~45 second runtime
ufcFights = []

# put all the events in the ufcFights dataset 
for url in events:
  row = events[url]
  ufcFights.append(row)  

In [ ]:

# "Fights" is the result of running concurrent program to scrape all fights on the site
fights = concurrent_ufc_fights(ufcFighterLinks, 8) # use 8 workers for ~45 second runtime
ufcFights = []

# put all the events in the ufcFights dataset 
for url in events:
  row = events[url]
  ufcFights.append(row)  
